# Interactive Map

The goal is to be a able to find on a map the nearest or the most relevant public toilet in Paris.
It will be done with Python and Folium. The data has been gathered on open data sites of Paris and RATP. 

In [59]:
import pandas as pd 
import numpy as np 
import folium

In [60]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import requests

from math import sin, cos, sqrt, atan2, radians

In [61]:
df = pd.read_csv('paris_sanisette.csv')

In [62]:
class Customer:
    
    def __init__(self, position, is_pmr, mode):
        self.position = position
        self.is_pmr = is_pmr
        self.mode = mode

In [72]:
def ColorsForPMR(df):
    colors = []
    for i in range(len(df)):
        if df['ACCES_PMR'].iloc[i] == ('oui' or 'Oui'):
            colors.append('green')
        else:
            colors.append('red')
    return colors 

In [73]:
def AllToiletsAMap(df):
    
    data = df
    
    paris_coord = [48.866667 , 2.333333]
    toilet_map = folium.Map(location=paris_coord, zoom_start = 12)
    
    marker_cluster = MarkerCluster()
    toilet_map.add_child(marker_cluster)

    data['colors'] = ColorsForPMR(data)

    for i, record in data.iterrows():
        marker = folium.Marker(location = (data.iloc[i]['LAT'],data.iloc[i]['LONG']), 
                               icon = folium.Icon(color='white', icon_color =data.iloc[i]['colors'])
                              ).add_to(marker_cluster)
        
        marker_cluster.add_child(marker)
        
    return toilet_map


In [74]:
AllToiletsAMap(df)

In [75]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [110]:
def NearestToilet(customer, df):
    
    z = 0
    for i in range(len(df)):
        distance = calculate_distance(customer.position[0], customer.position[1], 
                                      df['LAT'].iloc[i], df['LONG'].iloc[i])
        if i == 0:
            lowest_distance = distance
        if distance <= lowest_distance:
            lowest_distance = distance
            z = i
    
    nearest_toilet = df.iloc[z]
    
    return nearest_toilet, lowest_distance

def get_directions_response(lat1, long1, lat2, long2, mode='drive', key='zeze'):
        url = "https://route-and-directions.p.rapidapi.com/v1/routing"
        key = key
        host = "route-and-directions.p.rapidapi.com"
        headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": host}
        querystring = {"waypoints":f"{str(lat1)},{str(long1)}|{str(lat2)},{str(long2)}","mode":mode}
        response = requests.request("GET", url, headers=headers, params=querystring)
        return response

In [111]:
def create_map(response):
   # use the response
        mls = response.json()['features'][0]['geometry']['coordinates']
    
        points = [(i[1], i[0]) for i in mls[0]]
        m = folium.Map()
        # add marker for the start and ending points
        for point in [points[0], points[-1]]:
            folium.Marker(point).add_to(m)
        # add the lines
        folium.PolyLine(points, weight=5, opacity=1).add_to(m)
        # create optimal zoom
        df = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
        sw = df[['Lat', 'Lon']].min().values.tolist()
        ne = df[['Lat', 'Lon']].max().values.tolist()
        m.fit_bounds([sw, ne])
        return m

In [118]:
def INeedToPee(customer, df):
    
    data = df 
    
    # handi persons condition 
    if customer.is_pmr == True: data = data[(data['ACCES_PMR'] == 'oui') | data['ACCES_PMR'] == 'Oui']
    
    nearest_toilet, dist = NearestToilet(customer, data)
    
    # getting the direction
    #response = get_directions_response(customer.position[0], customer.position[1], nearest_toilet['LAT'], nearest_toilet['LONG'], api_key)
    response = get_directions_response(customer.position[0], customer.position[1], 
                                       nearest_toilet['LAT'], nearest_toilet['LONG'], 
                                       mode=customer.mode, key="key_is_hidden_from_public_display")
    
    m = create_map(response)
    return m


In [119]:
c1 = Customer(position=[48.862725, 2.287592], is_pmr=False, mode ='walk')

In [120]:
INeedToPee(c1, df)

{'features': [{'type': 'Feature', 'properties': {'mode': 'walk', 'waypoints': [{'location': [2.287592, 48.862725], 'original_index': 0}, {'location': [2.289051, 48.862588], 'original_index': 1}], 'units': 'metric', 'distance': 169, 'distance_units': 'meters', 'time': 173.65, 'legs': [{'distance': 169, 'time': 173.65, 'steps': [{'from_index': 0, 'to_index': 3, 'distance': 80, 'time': 68.89, 'instruction': {'text': 'Walk southeast on the walkway.'}}, {'from_index': 3, 'to_index': 4, 'distance': 2, 'time': 1.756, 'instruction': {'text': 'Continue.'}}, {'from_index': 4, 'to_index': 5, 'distance': 8, 'time': 7.024, 'instruction': {'text': 'Continue on the walkway.'}}, {'from_index': 5, 'to_index': 6, 'distance': 3, 'time': 2.634, 'instruction': {'text': 'Continue.'}}, {'from_index': 6, 'to_index': 7, 'distance': 10, 'time': 7.024, 'instruction': {'text': 'Continue on the walkway.'}}, {'from_index': 7, 'to_index': 10, 'distance': 36, 'time': 46.044, 'instruction': {'text': 'Turn left onto th